## Pose Estimation Models

references:
- stacked hourglass networks for human pose estimation: [https://arxiv.org/abs/1603.06937](https://arxiv.org/abs/1603.06937)
- simple baselines for human pose estimation and tracking: [https://arxiv.org/abs/1804.06208](https://arxiv.org/abs/1804.06208)
- [https://curt-park.github.io/2018-07-03/stacked-hourglass-networks-for-human-pose-estimation/](https://curt-park.github.io/2018-07-03/stacked-hourglass-networks-for-human-pose-estimation/)
- Deconvolution(Transposed Convolution) [https://zzsza.github.io/data/2018/06/25/upsampling-with-transposed-convolution/)

code references: 
- [https://github.com/bearpaw/pytorch-pose](https://github.com/bearpaw/pytorch-pose)
- [https://github.com/princeton-vl/pose-hg-train](https://github.com/princeton-vl/pose-hg-train)
- [https://github.com/microsoft/human-pose-estimation.pytorch](https://github.com/microsoft/human-pose-estimation.pytorch)

dataset annotatation references:
- [https://github.com/bearpaw/pytorch-pose](https://github.com/bearpaw/pytorch-pose)
- [https://github.com/HRNet/HRNet-Human-Pose-Estimation](https://github.com/HRNet/HRNet-Human-Pose-Estimation)

### Stacked Hourglass Networks for Human Pose Estimation (2016)

#### Multi-Stage Architecture

<img src="./src_imgs/10.png" width=600><br/>

<img src="./src_imgs/9.jpg"><br/>

"stacked hourglass networks for human pose estimation" 논문에서 제시한 Stacked Hourglass는 말그대로 모래시계 모양의 동일한 Network를 여러겹 쌓아서 만든 모델입니다. 이렇게 단일 네트워크를 여러겹 쌓는 구조는 Pose Estimation 분야에서 종종 볼 수 있습니다. 그 이유는 각 Stack마다 heatmap을 출력해보면 알 수 있는데, 2번째 그림과 같이 결과를 refine 시킬 수 있다는 장점을 갖기 때문입니다. 이러한 구조를 Multi-Stage Architecture라 하며 Single-Stage Architecture보다 정밀한 결과를 얻을 것이라는 아이디어에서 출발하였습니다. 

#### Multi-Scale

|![](./src_imgs/6.png)|
|:---:|
|*Hourglass*|

|![](./src_imgs/7.png)|
|:---:|
|*Stacked Hourglass (two-stack)*|

Stacked Hourglass가 포즈를 추출하는 것에 있어 우수한 성능을 내는 또 하나의 이유는 이미지의 모든 scale에서 정보를 얻을 수 있기 때문입니다. 단일 Hourglass network는 high resolution에서 low resolution으로 features를 만들어내는 **Downsampling Process**, 다시 low resolution에서 high resolution으로 resolution을 복구시키는 **Upsampling Process**를 통해 다양한 scale에서 정보를 얻습니다 (e.g. 64->32->16->8->4, 4->8->16->32->64). 

#### Residual Block / Intermediate Supervision

|![](./src_imgs/11.png)|
|:---:|
|*Residual Block*|

|![](./src_imgs/12.png)|
|:---:|
|*Intermediate Supervision*|

Multi-Stage 구조의 가장 큰 문제는 네트워크가 쌓일수록 layer가 깊어져서 Deep Neural Net의 고질적인 문제인 vanishing gradient(깊은 layer에서는 gradient의 영향력이 작아지는 문제)와 degradation(얕은 layer보다 깊은 layer를 갖는 모델이 오히려 성능이 안좋아지는 문제)이 발생한다는 점입니다. 
이것을 Stacked Hourglass는 residual block과 intermediate supervision을 차용함으로 학습 문제를 개선하였습니다.

Residual block의 특징은 module의 input과 output(convolution + activation function 결과)을 element-wise addition한다는 것입니다. 이러한 구조는 gradient를 유지하기 쉽게 만들어주기 때문에 vanishing gradient을 개선할 수 있는 장점이 존재합니다.

Intermediate supervision이란 각 stack 끝마다 loss layer를 추가하는 것으로 gradient를 비효율적이지만 효과적으로 전달하는 방법입니다. 깊은 stack일수록 gradient의 값이 0에 가까우므로 gradient를 깊은 레이어까지 직접 전달하는 것으로 해결하였습니다.


#### 모델 정의 및 구조 시각화

이제 Hourglass 모델을 정의하고 stack=2 만큼 쌓은 네트워크의 구조를 시각화하겠습니다.

Hourglass 모델을 이루는 기본 block은 앞서 말했듯이 Residual block을 사용합니다.<br/>
Block의 입력인 x와 컨볼루션 결과인 out이 함께 더해져서 연결되는 모습(skip connection)을 확인할 수 있습니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# residual block
class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()

        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 2, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out
        

위에서 정의한 residual block은 다음 hourglass module에서 기본 block으로 사용됩니다. 

hourglass에선 다양한 scale의 resolution에서 정보를 얻기 위해 MaxPooling을 이용해 downsampling 하고, 최근접 이웃보간을 통해 upsampling 합니다 (`F.max_pool2d()`, `F.interpolate()`).

In [2]:
class Hourglass(nn.Module):
    def __init__(self, block, num_blocks, planes, depth):
        super(Hourglass, self).__init__()
        self.depth = depth
        self.block = block
        self.hg = self._make_hour_glass(block, num_blocks, planes, depth)

    def _make_residual(self, block, num_blocks, planes):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(planes*block.expansion, planes))
        return nn.Sequential(*layers)

    def _make_hour_glass(self, block, num_blocks, planes, depth):
        hg = []
        for i in range(depth):
            res = []
            for j in range(3):
                res.append(self._make_residual(block, num_blocks, planes))
            if i == 0:
                res.append(self._make_residual(block, num_blocks, planes))
            hg.append(nn.ModuleList(res))
        return nn.ModuleList(hg)

    def _hour_glass_forward(self, n, x):
        up1 = self.hg[n-1][0](x)
        low1 = F.max_pool2d(x, 2, stride=2)
        low1 = self.hg[n-1][1](low1)

        if n > 1:
            low2 = self._hour_glass_forward(n-1, low1)
        else:
            low2 = self.hg[n-1][3](low1)
        low3 = self.hg[n-1][2](low2)
        up2 = F.interpolate(low3, scale_factor=2) # Nearest Neighbour
        out = up1 + up2
        return out

    def forward(self, x):
        return self._hour_glass_forward(self.depth, x)
        

이제 Hourglass 모듈을 stack으로 쌓을 수 있는 네트워크를 정의합니다.

`HourglassNet`은 hourglass 스택 개수(`num_stack`), 한 block에 포함된 residual block 개수(`num_block`), parts 개수(`num_classes`)를 조정함으로 모델의 크기를 정합니다. 

In [3]:
 class HourglassNet(nn.Module):
    '''Hourglass model from Newell et al ECCV 2016'''
    def __init__(self, block, num_stacks=2, num_blocks=4, num_classes=16):
        super(HourglassNet, self).__init__()

        self.inplanes = 64
        self.num_feats = 128
        self.num_stacks = num_stacks
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=True)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_residual(block, self.inplanes, 1)
        self.layer2 = self._make_residual(block, self.inplanes, 1)
        self.layer3 = self._make_residual(block, self.num_feats, 1)
        self.maxpool = nn.MaxPool2d(2, stride=2)

        # build hourglass modules
        ch = self.num_feats*block.expansion
        hg, res, fc, score, fc_, score_ = [], [], [], [], [], []
        for i in range(num_stacks):
            hg.append(Hourglass(block, num_blocks, self.num_feats, 4))
            res.append(self._make_residual(block, self.num_feats, num_blocks))
            fc.append(self._make_fc(ch, ch))
            score.append(nn.Conv2d(ch, num_classes, kernel_size=1, bias=True))
            if i < num_stacks-1:
                fc_.append(nn.Conv2d(ch, ch, kernel_size=1, bias=True))
                score_.append(nn.Conv2d(num_classes, ch, kernel_size=1, bias=True))
        self.hg = nn.ModuleList(hg)
        self.res = nn.ModuleList(res)
        self.fc = nn.ModuleList(fc)
        self.score = nn.ModuleList(score)
        self.fc_ = nn.ModuleList(fc_)
        self.score_ = nn.ModuleList(score_)

    def _make_residual(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=True),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_fc(self, inplanes, outplanes):
        bn = nn.BatchNorm2d(inplanes)
        conv = nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=True)
        return nn.Sequential(
                conv,
                bn,
                self.relu,
            )

    def forward(self, x):
        out = []
        # preprocessing
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.maxpool(x)
        x = self.layer2(x)
        x = self.layer3(x)

        # stacking modules
        for i in range(self.num_stacks):
            y = self.hg[i](x)
            y = self.res[i](y)
            y = self.fc[i](y)
            score = self.score[i](y)
            out.append(score)
            if i < self.num_stacks-1:
                fc_ = self.fc_[i](y)
                score_ = self.score_[i](score)
                x = x + fc_ + score_

        return out
    

Stack=1인 HourglassNet의 구조를 출력해보면 다음과 같습니다. 

레이어가 깊어질수록 output resolution이 128->64->32->16->8->4 까지 줄어들고, 
다시 4->8->16->32->64 까지 증가함을 볼 수 있습니다. 또한, 마지막 레이어에선 heatmap([1, 16, 64, 64])을 출력하고 있습니다. 

In [4]:
from torchsummary import summary

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# hourglass
model = HourglassNet(Bottleneck, num_stacks=1, num_blocks=1, num_classes=16).to(device)

summary(model, input_size=(3, 256, 256), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           9,472
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
              ReLU-4          [1, 64, 128, 128]               0
       BatchNorm2d-5          [1, 64, 128, 128]             128
              ReLU-6          [1, 64, 128, 128]               0
            Conv2d-7          [1, 64, 128, 128]           4,160
       BatchNorm2d-8          [1, 64, 128, 128]             128
              ReLU-9          [1, 64, 128, 128]               0
           Conv2d-10          [1, 64, 128, 128]          36,928
      BatchNorm2d-11          [1, 64, 128, 128]             128
             ReLU-12          [1, 64, 128, 128]               0
           Conv2d-13         [1, 128, 128, 128]           8,320
           Conv2d-14         [1, 128, 1

In [5]:
# 2 stacked hourglass
model = HourglassNet(Bottleneck, num_stacks=2, num_blocks=1, num_classes=16).to(device)

summary(model, input_size=(3, 256, 256), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           9,472
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
              ReLU-4          [1, 64, 128, 128]               0
              ReLU-5          [1, 64, 128, 128]               0
       BatchNorm2d-6          [1, 64, 128, 128]             128
              ReLU-7          [1, 64, 128, 128]               0
            Conv2d-8          [1, 64, 128, 128]           4,160
       BatchNorm2d-9          [1, 64, 128, 128]             128
             ReLU-10          [1, 64, 128, 128]               0
           Conv2d-11          [1, 64, 128, 128]          36,928
      BatchNorm2d-12          [1, 64, 128, 128]             128
             ReLU-13          [1, 64, 128, 128]               0
           Conv2d-14         [1, 128, 1

### Simple Baselines for Human Pose Estimation and Tracking (2018)

#### Single-Stage Architecture

<img src="./src_imgs/13.png" width=800>

한편, "Simple Baselines for Human Pose Estimation and Tracking" 논문의 저자들은 기존 Multi-Stage 구조의 모델들의 알고리즘적인 복잡성에 주목했습니다. 기존 Multi-Stage 의 비효율적이고 파악하기 어려운 모델 구조는 네트워크를 설계하는 것에 있어 복잡도를 크게 증가시켰습니다. 이러한 점에서 저자는 Backbone Network에 Deconvolution layer와 간단한 Upsampling을 추가하는 것만으로도 기존 모델들에 버금가는 성능을 가질 수 있음을 제시하였습니다. 

위 그림에서 (a)는 Hourglass 네트워크를 나타내는데, 저자가 제시한 (c) 모델에 비해 상당히 간단한 구조를 갖습니다.

#### Backbone Network

<img src="./src_imgs/14.png" height=400>

저자의 목표는 "간단한 방법이 얼마나 좋은 성능을 낼 수 있는가?" 에 대한 답을 얻는 것이었습니다. 저자는 대규모 이미지 데이터셋인 imagenet에서 classification task를 위해 훈련된 ResNet을 Backbone으로 사용하였습니다. 앞서 Hourglass에서 사용한 Residual Block은 사실 ResNet에서 처음 제시되었습니다. 저자는 ResNet을 이용해 이미지의 특징을 추출하면 이러한 features로부터 포즈 역시 잘 파악할 수 있다는 전제하에 사용하였습니다. 실제로 대규모 데이터셋에서 학습된 네트워크를 feature extractor (or backbone)로써 사용하는 경우가 많으며, 비슷한 task에서 좋은 성능을 보입니다. 

imagenet에서 학습된 ResNet 네트워크는 가장 마지막 layer로 classification을 수행하는 fully connected layer (fc layer)를 가집니다. 논문에서는 feature extractor로써만 사용하기 위해 fc layer는 떼어내어 버리고 제안된 layer만 붙이는 구조를 제안합니다.

#### Deconvolution layer

|![](https://cdn-images-1.medium.com/max/1200/1*BMngs93_rm2_BpJFH2mS0Q.gif)|
|:---:|
|*2D convolution with no padding, stride of 2 and kernel of 3*|

|![](https://cdn-images-1.medium.com/max/1200/1*Lpn4nag_KRMfGkx1k6bV-g.gif)|
|:---:|
|*Transposed 2D convolution with no padding, stride of 2 and kernel of 3*|


논문에선 Deconvolution 이라고 언급하고 있지만 실제 연산은 Deconvolution이 아닌 Transposed Convolution을 수행합니다. Deconvolution은 사실 Upsampling을 수행하기 위해 Convolution의 역연산 과정을 취한 것입니다. 반면, Transposed Convolution은 Deconvolution과 동일한 Resolution을 복구할 뿐 수학적인 연산 자체는 다릅니다. 이미지의 다양한 scale에서 정보를 추출하기 위한 Upsampling 방법 중 하나로 Convolution과 같이 학습 가능한 parameters가 존재합니다. 

앞서 Hourglass에선 Upsampling 과정으로 Nearest Neighbour Interpolation을 사용한 반면, 저자는 Deconvolution (Transposed Convolution)으로 수행했다는 점 정도로 기억하면 좋을것 같습니다. 

더 자세한 내용은 Reference 에 관련 링크를 참고하세요.


#### 모델 정의 및 구조 시각화

이제 ResNet-101 Backbone을 사용하여 Simple Baseline 모델을 정의하겠습니다.

먼저, ResNet-101에서 사용하는 Residual Block을 정의합니다. 

In [1]:
import os
import logging

import torch
import torch.nn as nn
from collections import OrderedDict


BN_MOMENTUM = 0.1
logger = logging.getLogger(__name__)


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion,
                                  momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

ResNet-101과 Upsampling process (Deconvolution)를 통합한 전체 네트워크를 정의합니다. 
`forward()`를 보면 ResNet을 거쳐 마지막에 Deconvolution layer와 final layer만 추가한 것을 확인할 수 있습니다.
final layer는 1x1 convolution입니다. output channel을 joints 개수로 맞추어 heatmap을 생성함을 알 수 있습니다. 

In [2]:
resnet_spec = {101: (Bottleneck, [3, 4, 23, 3])}


def get_pose_net(cfg, is_train, **kwargs):
    num_layers = cfg.MODEL.EXTRA.NUM_LAYERS

    block_class, layers = resnet_spec[num_layers]

    model = PoseResNet(block_class, layers, cfg, **kwargs)

    return model


class PoseResNet(nn.Module):

    def __init__(self, block, layers, cfg, **kwargs):
        self.inplanes = 64
        extra = cfg.MODEL.EXTRA
        self.deconv_with_bias = extra.DECONV_WITH_BIAS

        super(PoseResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # used for deconv layers
        self.deconv_layers = self._make_deconv_layer(
            extra.NUM_DECONV_LAYERS,
            extra.NUM_DECONV_FILTERS,
            extra.NUM_DECONV_KERNELS,
        )

        self.final_layer = nn.Conv2d(
            in_channels=extra.NUM_DECONV_FILTERS[-1],
            out_channels=cfg.MODEL.NUM_JOINTS,
            kernel_size=extra.FINAL_CONV_KERNEL,
            stride=1,
            padding=1 if extra.FINAL_CONV_KERNEL == 3 else 0
        )

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _get_deconv_cfg(self, deconv_kernel, index):
        if deconv_kernel == 4:
            padding = 1
            output_padding = 0
        elif deconv_kernel == 3:
            padding = 1
            output_padding = 1
        elif deconv_kernel == 2:
            padding = 0
            output_padding = 0

        return deconv_kernel, padding, output_padding

    def _make_deconv_layer(self, num_layers, num_filters, num_kernels):
        assert num_layers == len(num_filters), \
            'ERROR: num_deconv_layers is different len(num_deconv_filters)'
        assert num_layers == len(num_kernels), \
            'ERROR: num_deconv_layers is different len(num_deconv_filters)'

        layers = []
        for i in range(num_layers):
            kernel, padding, output_padding = \
                self._get_deconv_cfg(num_kernels[i], i)

            planes = num_filters[i]
            layers.append(
                nn.ConvTranspose2d(
                    in_channels=self.inplanes,
                    out_channels=planes,
                    kernel_size=kernel,
                    stride=2,
                    padding=padding,
                    output_padding=output_padding,
                    bias=self.deconv_with_bias))
            layers.append(nn.BatchNorm2d(planes, momentum=BN_MOMENTUM))
            layers.append(nn.ReLU(inplace=True))
            self.inplanes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.deconv_layers(x)
        x = self.final_layer(x)

        return x


이제 ResNet-101을 Backbone으로 사용하는 PoseResNet의 구조를 출력해보겠습니다.

PoseResNet이 사용하는 Backbone인 ResNet 역시 가벼운 네트워크는 아닙니다.
제안된 방법은 단순히 파라미터를 줄이려는 것이 아니라, 기존의 down, up sampling을 반복적으로 해야한다는 기틀에서 벗어나 마지막 단에만 Upsampling 과정을 추가하는 것으로 좋은 성능을 낼 수 있다는 점을 보였습니다.

In [10]:
from torchsummary import summary
from easydict import EasyDict as edict

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

cfg = edict()

cfg.MODEL = edict()
cfg.MODEL.NUM_JOINTS = 16

cfg.MODEL.EXTRA = edict()
cfg.MODEL.EXTRA.NUM_LAYERS = 101
cfg.MODEL.EXTRA.NUM_DECONV_KERNELS = [4, 4, 4]
cfg.MODEL.EXTRA.NUM_DECONV_FILTERS = [256, 256, 256]
cfg.MODEL.EXTRA.NUM_DECONV_LAYERS = 3
cfg.MODEL.EXTRA.DECONV_WITH_BIAS = False
cfg.MODEL.EXTRA.FINAL_CONV_KERNEL = 1

model = get_pose_net(cfg=cfg, is_train=False)
model = model.to(device)

summary(model=model, input_size=(3, 256, 256), batch_size=1)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           9,408
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
         MaxPool2d-4            [1, 64, 64, 64]               0
            Conv2d-5            [1, 64, 64, 64]           4,096
       BatchNorm2d-6            [1, 64, 64, 64]             128
              ReLU-7            [1, 64, 64, 64]               0
            Conv2d-8            [1, 64, 64, 64]          36,864
       BatchNorm2d-9            [1, 64, 64, 64]             128
             ReLU-10            [1, 64, 64, 64]               0
           Conv2d-11           [1, 256, 64, 64]          16,384
      BatchNorm2d-12           [1, 256, 64, 64]             512
           Conv2d-13           [1, 256, 64, 64]          16,384
      BatchNorm2d-14           [1, 256,